**Exercise 8** 
This is the code to compute the likelihood function of exercise 8 of problem set 3. 


In [22]:
using StatFiles, DataFrames

data_path = "/Users/jorditorresvallverdu/Downloads/xsection_05.dta"

export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/policy_effects/figures/"


main = DataFrame(load(data_path))

Row,occup,d_months,hwage
,Float32?,Float32?,Float32?
1,3.0,10.0,47.619
2,2.0,1.0,18.4854
3,2.0,7.0,22.5694
4,3.0,244.0,21.3294
5,3.0,88.0,7.93651
6,3.0,124.0,16.7143
7,3.0,136.0,21.645
8,1.0,0.0,missing
9,3.0,2.0,25.5952


In [23]:
##Code tbe parameters of the likelihood function
using Distributions

function log_likelihood(θ::AbstractVector, w_i_star::AbstractFloat,w_f_star::AbstractFloat,t_u::AbstractVector, w_i::AbstractVector, w_f::AbstractVector,u::AbstractVector, e_i::AbstractVector, e_f::AbstractVector)

μ_i, μ_f, σ_i, σ_f, λ_i, λ_f, η_i, η_f = unpack_params(θ)


Fi_star= cdf.(LogNormal(μ_i, σ_i), w_i_star) #need to define w_i_star properly following Heckman
Ff_star= cdf.(LogNormal(μ_f, σ_f), w_f_star) #need to define w_f_star outside too

F_i_tilde=  1-Fi_star
F_f_tilde=  1-Ff_star

a_i=  λ_i* F_i_tilde
a_f=  λ_f* F_f_tilde

h_u=  a_i + a_f

p_u= (1+(a_i/η_i)+(a_f/η_f))^-1
p_ei= (a_i/η_i)/(1+(a_i/η_i)+(a_f/η_f))
p_ef=  (a_f/η_f)/(1+(a_i/η_i)+(a_f/η_f))

f_i= pdf.(LogNormal(μ_i, σ_i), w_i) ##need to define w_i as input
f_f= pdf.(LogNormal(μ_f, σ_f), w_f)


# Create full-length zero vectors
f_i_full = zeros(length(u))
f_f_full = zeros(length(u))

# Fill only where sectors match
f_i_full[e_i] .= f_i
f_f_full[e_f] .= f_f

ll_u= @. u*(log(h_u) - h_u* t_u+ log(p_u))
ll_ei= @. e_i *(log(f_i_full)- log(F_i_tilde)+ log(p_ei)) 
ll_ef= @. e_f*((log(f_f_full))- log(F_f_tilde)+ log(p_ef))


    return -(sum(ll_u) + sum(ll_ef) + sum(ll_ei))
end


log_likelihood (generic function with 2 methods)

In [24]:
##define parameters/inputs of the function 
u= (main.occup .== 1) #vector
e_f= (main.occup .==2) #vector
e_i= (main.occup .==3) #vector

t_u= main.d_months #vector


w_i=main.hwage[e_i] #vector
w_f=main.hwage[e_f] #vector

w_i_star= minimum(w_i) #scalar
w_f_star= minimum(w_f) #scalar



5.7963266f0

In [25]:
function unpack_params(θ)

    μ_i = θ[1]
    μ_f = θ[2]

    σ_i = exp(θ[3])
    σ_f = exp(θ[4])

    λ_i = exp(θ[5])
    λ_f = exp(θ[6])

    η_i = exp(θ[7])
    η_f = exp(θ[8])

    return μ_i, μ_f, σ_i, σ_f, λ_i, λ_f, η_i, η_f
end

unpack_params (generic function with 1 method)

In [26]:
θ0 = [
    mean(log.(w_i)),             # μ_i
    mean(log.(w_f)),             # μ_f
    log(std(log.(w_i))),         # log σ_i
    log(std(log.(w_f))),         # log σ_f
    log(0.2),                    # log λ_i
    log(0.2),                    # log λ_f
    log(0.05),                   # log η_i
    log(0.05)                    # log η_f
]

obj(θ) = log_likelihood(θ,
    w_i_star, w_f_star,
    t_u, w_i, w_f,
    u, e_i, e_f
)

obj (generic function with 1 method)

In [27]:
##Optimize
using Optim

res = optimize(obj, θ0, BFGS())
θ_hat = Optim.minimizer(res)
LL_min = Optim.minimum(res)

println("Estimated parameters θ̂ = ", θ_hat)
println("Minimum negative log-likelihood = ", LL_min)


Estimated parameters θ̂ = [2.88049037674668, 3.13613428593826, -0.5454886684093873, -0.6474618104028487, -1.4935711294875555, -1.9297163961710022, -3.8211165203634305, -4.388982369432443]
Minimum negative log-likelihood = 69290.93898062492
